In [6]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import LeakyReLU
import tensorflow as tf
from keras import metrics

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

In [17]:
# Building CNN

cnn = Sequential()

# Convolution
cnn.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
cnn.add(LeakyReLU(0.1))

# Pooling
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd Convolution
cnn.add(Conv2D(32, (3, 3)))
cnn.add(LeakyReLU(0.1))

# 2nd Pooling layer
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# # 3rd Convolution
cnn.add(Conv2D(32, (2, 2), activation="relu"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# # # 4th layer
cnn.add(Conv2D(32, (2, 2), activation="relu"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# Flatten the layer
cnn.add(Flatten())

# Fully Connected Layers
cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'sigmoid', units = 1))

# Compile the Neural network

METRICS = [
                'accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
        ] 


cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = METRICS)

In [18]:
# Fitting the CNN to the images

trainDataGenerator = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

testDataGenerator = ImageDataGenerator(rescale = 1./255)

# Reducing the image size to make it more managable
# Jakobs directory: /content/drive/MyDrive/Machine Learning Final Project/Data/chest_xray/
trainingData = trainDataGenerator.flow_from_directory('drive/MyDrive/xray/chest_xray/train', target_size = (64, 64), batch_size = 32, class_mode = 'binary')

validationData = testDataGenerator.flow_from_directory('drive/MyDrive/xray/chest_xray/val', target_size=(64, 64), batch_size=32, class_mode='binary')

testData = testDataGenerator.flow_from_directory('drive/MyDrive/xray/chest_xray/test', target_size = (64, 64), batch_size = 32, class_mode = 'binary')


Found 5219 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [19]:
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 29, 29, 32)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 32)       

In [20]:
cnn_model = cnn.fit(trainingData, steps_per_epoch = 100, epochs = 1, validation_data = validationData, validation_steps = 600)

163/163 [==============================] - 60s 365ms/step - loss: 0.5258 - accuracy: 0.7544 - precision: 0.7584 - recall: 0.9815 - val_loss: 0.6313 - val_accuracy: 0.6250 - val_precision: 0.5714 - val_recall: 1.0000


In [ ]:
test_accu = cnn.evaluate(testData,steps=624)

In [ ]:
print('The testing accuracy is :',test_accu[1]*100, '%')

In [ ]:
M2_3Layer_Data = [80.9, 82.2, 84.5, 78.2, 75.8, 85.7, 73.6, 85.9]
M2_4Layer_Data = [78.2, 63.6, 64.7, 82.21, 70.9, 76.3, 72.4, 71.47]
M1_Data = [83.45, 85.09, 81.6, 87.18, 85.4, 85.7, 77.24, 81.09]
M3_Sigmoid_Data = [62.5, 62.5, 62.5, 62.5, 62.5, 62.5, 62.5, 62.5]
M3_LeakReLU_Data = [79.48, 82.05, 87.5, 83.5, 79.6, 79.0, 81.7, 84.1]
trialCount = [1, 2, 3, 4, 5, 6, 7, 8]
def plot_data(xData, yData, mainTitle, xTitle, yTitle, legendTitle, color):
  plt.plot(xData, yData, label=legendTitle, color=color)
  plt.title(mainTitle)
  plt.xlabel(xTitle)
  plt.ylabel(yTitle)
  plt.legend(bbox_to_anchor=(1, 1))

plot_data(trialCount, M2_3Layer_Data, "Test Accuracies", "Trial #", "Accuracy(%)", "Method 2 Accuracies: 3 Layers", "blue")
plot_data(trialCount, M2_4Layer_Data, "Test Accuracies", "Trial #", "Accuracy(%)", "Method 2 Accuracies: 4 Layers", "orange")
plot_data(trialCount, M1_Data, "Test Accuracies", "Trial #", "Accuracy(%)", "Method 1 Test Accuracies", "red")
plot_data(trialCount, M3_Sigmoid_Data, "Test Accuracies", "Trial #", "Accuracy(%)", "Method 3 Accuracies: Sigmoid", "green")
plot_data(trialCount, M3_LeakReLU_Data, "Test Accuracies", "Trial #", "Accuracy(%)", "Method 3 Accuracies: Leaky ReLU", "violet")

plt.show()



def plot_bar_graph(Title, xtitle, ytitle):

  methods = ['Method 1', 'Method 2: 3 Layers', 'Method 2: 4 Layers', 'Method 3: Sigmoid', 'Method 4: LeakyReLU']
  xMethods = ['M1', 'M2.3L', 'M2.4L', 'M3.SIG', 'M4.L-RELU']

  average_times = [61.6, 60.1, 58.0, 59.0, 59.0]

  plt.bar(xMethods, average_times, width=0.35, color=['black', 'red', 'green', 'blue', 'cyan'])
  plt.xlabel(xtitle)
  plt.ylabel(ytitle)
  plt.title(Title)
  plt.show()



plot_bar_graph("Method Computation Times", "Method", "Computation Time (s)")
